In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
holidayDataDenver = pd.read_csv('./Data/Denver_Holiday.csv')
holidayDataVancouver = pd.read_csv('./Data/Vancouver_Holiday.csv')
eventDataDenver = pd.read_csv('./Data/Denver_events.csv')
eventDataVancouver = pd.read_csv('./Data/Vancouver_events.csv')

In [ ]:
date_df = pd.DataFrame(columns=['date_key', 'year','month','day', 'day_of_week','is_weekend','is_holiday_vancouver','holiday_name_vancouver','is_holiday_denver','holiday_name_denver'])

#Full Date Dimesion
start='20150101'
end='20191231'
s = pd.Series(pd.date_range(start, end))
date_df["date_key"] = s.dt.strftime('%Y%m%d')
date_df["day_of_week"] = s.dt.weekday_name
date_df['is_weekend'] = ((pd.to_numeric( s.dt.weekday)) // 5 == 1).astype('int64')
date_df["year"] = s.dt.year
date_df["month"] = s.dt.month
date_df["day"] = s.dt.day
date_df["is_holiday_denver"] = 0
date_df["is_holiday_vancouver"] = 0
#Put Denver Holiday csv into Date Dimension
for idx_hol, row_hol in holidayDataDenver.iterrows():
    for idx_date, row_date in date_df.iterrows():
        if row_hol['YEAR']==row_date['year'] and row_hol['MONTH']==row_date['month'] and row_hol['DAY']== row_date['day']:
            date_df.at[idx_date, 'is_holiday_denver'] = 1
            date_df.at[idx_date, 'holiday_name_denver'] = row_hol['NAME']
#Put Vancouver Holiday csv into Date Dimension
for idx_hol, row_hol in holidayDataVancouver.iterrows():
    for idx_date, row_date in date_df.iterrows():
        if row_hol['YEAR']==row_date['year'] and row_hol['MONTH']==row_date['month'] and row_hol['DAY']== row_date['day']:
            date_df.at[idx_date, 'is_holiday_vancouver'] = 1
            date_df.at[idx_date, 'holiday_name_vancouver'] = row_hol['NAME']

In [ ]:
event_denver_df = pd.DataFrame(columns=['event_key','event_name','event_type','event_location','event_neighbourhood','event_city','event_start_date','event_end_date','event_location_size'])
#Event Dimension only has Denver Event
for idx_event, row_event in eventDataDenver.iterrows():
    event_row=[idx_event+3000000]
    event_row += [row_event['NAME'],row_event['TYPE'],row_event['LOCATION'],row_event['NEIGHBOURHOOD'],'Denver',row_event['DATE_START'],row_event['DATE_END'],row_event['LOCATION-SIZE']]
    event_denver_df.loc[len(event_denver_df)] = event_row

In [ ]:
#Event Dimension
event_vancouver_df = pd.DataFrame(columns=['event_key','event_name','event_type','event_location','event_neighbourhood','event_city','event_start_date','event_end_date','event_location_size'])
for idx_event, row_event in eventDataVancouver.iterrows():
    event_row=[len(event_denver_df)+3000000]
    event_row += [row_event['NAME'],row_event['TYPE'],row_event['LOCATION'],row_event['NEIGHBOURHOOD'],'Vancouver',row_event['DATE_START'],row_event['DATE_END'],row_event['LOCATION-SIZE']]
    event_vancouver_df.loc[len(event_vancouver_df)] = event_row

In [ ]:
event_df = pd.concat([event_denver_df, event_vancouver_df], ignore_index=True)

In [ ]:
# Save to csv
date_df.to_csv(r'.\Dimensions\date_dimension.csv',index=False)
event_df.to_csv(r'.\Dimensions\event_dimension.csv',index=False)